In [ ]:
import pandas as pd
headlines = pd.read_parquet("data/raw/headlines_long.parquet")
prices = pd.read_parquet("data/raw/djia_prices.parquet")

headlines["trading_date"] = pd.to_datetime(headlines["Date"])
prices["trading_date"] = pd.to_datetime(prices["trading_date"])

df = headlines.merge(
    prices,
    on="trading_date",
    how="inner"
)

prices = prices.sort_values("trading_date")
prices["close_t_plus_1"] = prices["close"].shift(-1)

df = df.merge(
    prices[["trading_date", "close_t_plus_1"]],
    on="trading_date",
    how="left"
)

df["return_t_plus_1"] = (
    df["close_t_plus_1"] - df["close"]
) / df["close"]

df = df.dropna(subset=["return_t_plus_1"])

df["label"] = (df["return_t_plus_1"] > 0).astype(int)

df[[
    "ticker",
    "trading_date",
    "headline_text",
    "return_t_plus_1",
    "label"
]].to_parquet("data/processed/model_table.parquet")
